In [ ]:
from helpers import *
from gaussfit import *

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

## Configuration

In [ ]:
# list of PMTs at 0 voltage form HV files + disconnected pmts: 290
offPMTs = [1, 111, 143, 166, 192, 230, 238, 254, 222, 302, 309, 340, 353, 290 ]
offChs = PMTid_to_channel(offPMTs)

In [ ]:
srcpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/calibrationdb/"
histpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/quick/"
figpath = "/exp/icarus/data/users/mvicenzi/pmt-calibration/figs/"

In [ ]:
def compare(file_old, file_new):
    file = srcpath + file_old 
    filen = srcpath + file_new

    run=getRun(file)
    data = getDataFrame(file, offPMTs, False)
    datan = getDataFrame(filen, offPMTs, False)
    
    plt.scatter(data["pmt"],data["q"],marker="o",label="{} - v09_67_00".format(run))
    plt.scatter(datan["pmt"],datan["q"],marker=".",color="red",label="{} - v09_77_00".format(run))
    plt.ylim((0.1,0.9))
    plt.xlabel("PMT channel ID")
    plt.ylabel("Gains [10^7 electrons]")
    plt.legend()
    plt.grid()
    plt.show()
    
    binsize=0.005
    b=int(0.9/binsize)
    plt.hist( data["q"], histtype="step", bins=b,range=(0.0,1.0),lw=2, label="{} - v09_67_00".format(run))
    plt.hist( datan["q"], histtype="step",lw=2, bins=b,range=(0.0,1.0), label="{} - v09_77_00".format(run))
    plt.xlim((0.1, 1.0))
    plt.ylabel("# PMTs")
    plt.xlabel("Gains [10^7 electrons]")
    plt.legend()
    plt.grid()
    plt.show()
    
    ## difference 
    df = data.join( datan, on=["pmt"], how="inner", rsuffix="_77")
    
    plt.scatter(df["pmt"],df['q']-df['q_77'],marker="o")
    plt.xlabel("PMT channel ID")
    plt.ylabel("G$_{v09.67.00}$ - G$_{v09.77.00}$ [10^7 electrons]")
    plt.ylim((-0.04,0.04))
    plt.grid()
    plt.show()
    
    plt.hist( df['q']-df['q_77'], histtype="step", bins=100,range=(-0.04,0.04),lw=2, label="All")
    plt.hist( df[df.pmt<180]['q']-df[df.pmt<180]['q_77'], histtype="step",lw=2, bins=100,range=(-0.04,0.04), label="East only")
    plt.hist( df[df.pmt>=180]['q']-df[df.pmt>=180]['q_77'], histtype="step",lw=2, bins=100,range=(-0.04,0.04), label="West only")
    plt.xlabel("G$_{v09.67.00}$ - G$_{v09.77.00}$ [10^7 electrons]")
    plt.ylabel("# PMTs")
    plt.legend()
    plt.grid()
    plt.show()
    
    plt.hist( (df['q']-df['q_77'])/df['q']*100, lw=2, histtype="step", bins=100, range=(-5,5), label="All")
    plt.hist( (df[df.pmt<180]['q']-df[df.pmt<180]['q_77'])/df[df.pmt<180]['q']*100, lw=2, histtype="step", bins=100, range=(-5,5), label="East only")
    plt.hist( (df[df.pmt>=180]['q']-df[df.pmt>=180]['q_77'])/df[df.pmt>=180]['q']*100, lw=2, histtype="step", bins=100, range=(-5,5), label="West only")
    plt.xlabel("(G$_{v09.67.00}$ - G$_{v09.77.00}$)/G$_{v09.67.00}$ [%]")
    plt.ylabel("# PMTs")
    plt.legend()
    plt.grid()
    plt.show()
    

## Run 10040

In [ ]:
file1 = "backgroundphotons_run10040_1688176609.csv" 
file2 = "test_v09_77_00_run10040_1688173090.csv"

compare(file1,file2)

## Run 10339

In [ ]:
file1 = "test_v09_67_00_run10339_1692403161.csv" 
file2 = "test_v09_77_00_run10339_1692402872.csv"

compare(file1,file2)

## Run 10336

In [ ]:
file1 = "equalization_run10336_1692384276.csv" 
file2 = "test_v09_77_00_run10336_1692384951.csv"

compare(file1,file2)

# COMPARE SINGLE CHANNELS

In [ ]:
import ROOT as r
from matplotlib.backends import backend_pdf

In [ ]:
def get_bins_edges(bin_centers):

    bin_edges = []
    bin_edges.append(0.0) #first bin left edge

    for i in range(len(bin_centers)):
        length = 0
        if i == (len(bin_centers)-1):
            length = bin_centers[i]-bin_centers[i-1]
        else:
            length = bin_centers[i+1]-bin_centers[i]
        bin_edges.append(bin_centers[i]+length/2.)

    #print(bin_edges)
    return bin_edges

def get_histo_things(hcharge, max, min):
    
    binned_events = []
    bin_centers = []
    for i in range(1,hcharge.GetNbinsX()+1):
        binned_events.append(hcharge.GetBinContent(i))
        bin_centers.append(hcharge.GetXaxis().GetBinCenter(i))
    bin_edges = get_bins_edges(bin_centers)
    
    fbin_centers = [x for x in bin_centers if x < max]
    fbin_edges = [x for x in bin_edges if x < max]
    length = len(fbin_centers)
    fbinned_events = binned_events[:length]
    
    return fbin_centers, fbin_edges, fbinned_events

def plot_all(pdf, filepath1, filepath2):
    file1 = r.TFile(filepath1,"READ")
    file2 = r.TFile(filepath2,"READ")
    
    dir1 = file1.Get("bkgcalibration")
    dir2 = file2.Get("bkgcalibration")

    for ch in range(0,360):
        
        hname = "hintegral"+str(ch)
        hamp = "hamplitude"+str(ch)

        hch1 = dir1.Get(hname)
        hch2 = dir2.Get(hname)
        hamp1 = dir1.Get(hamp)
        hamp2 = dir2.Get(hamp)
        
        ma = 2.0
        mi = 0.3
        c1, e1, w1 = get_histo_things(hch1,ma,mi)
        c2, e2, w2 = get_histo_things(hch2,ma,mi)
        
        ma = 10.0
        mi = 0.0
        a1, ea1, wa1 = get_histo_things(hamp1,ma,mi)
        a2, ea2, wa2 = get_histo_things(hamp2,ma,mi)
        
        fig, ax = plt.subplots(1,2,figsize=(12, 4))
        ax[0].hist(c1, bins=e1, weights=w1, histtype="step", label="Ch {} - v09_67_00\nEntries: {}".format(ch, int(hch1.GetEntries())), lw=1.5)
        ax[0].hist(c2, bins=e2, weights=w2, histtype="step", label="Ch {} - v09_77_00\nEntries: {}".format(ch, int(hch2.GetEntries())), lw=1.5)
        
        ax[0].grid(color='0.95')
        ax[0].set_xlabel(r'Pulse charge [$10^7$ electrons]')
        ax[0].set_ylabel("# events")
        ax[0].margins(x=0.00)
        ax[0].legend()
    
        ax[1].hist(a1, bins=ea1, weights=wa1, histtype="step", label="Ch {} - v09_67_00\nEntries: {}".format(ch, int(hamp1.GetEntries())), lw=1.5)
        ax[1].hist(a2, bins=ea2, weights=wa2, histtype="step", label="Ch {} - v09_77_00\nEntries: {}".format(ch, int(hamp2.GetEntries())), lw=1.5)
        
        ax[1].grid(color='0.95')
        ax[1].set_xlabel(r'Amplitude [mV]')
        ax[1].set_ylabel("# events")
        ax[1].margins(x=0.00)
        ax[1].legend()
    
        pdf.savefig( fig )
        plt.close()
    return pdf

In [ ]:
filepath1 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/histograms/pulseDistributionHist_run10040.root"
filepath2 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/histograms/pulseDistributionHist_run10040_v09_77_00.root"

#filepath1 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/histograms/pulseDistributionHist_run10339_v09_67_00.root"
#filepath2 = "/exp/icarus/data/users/mvicenzi/pmt-calibration/histograms/pulseDistributionHist_run10339_v09_77_00.root"

pdf = backend_pdf.PdfPages("pulse_charge_10040.pdf")
plot_all(pdf,filepath1,filepath2)
pdf.close()